### UKB phenotyping of patients with serious ADRs and WES available

In [1]:
%load_ext autoreload
%autoreload 2

# switch the directory to the main project directory
%cd /net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/

# wide notebook display in browser

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl

from analysis.utils.load_spark import spark_master_host, spark_master_port, localfs_path, scratch_path, hail_log_uuid, wd
from analysis.utils.load_spark import hl_init
from analysis.utils.annotations import (
    load_annotations,
    load_annotations_description,
    annotate_adr_patients,
    annotate_date_codes_dict,
    annotate_intent
)
from analysis.controls.kinship_pca_matches import remove_related, anno_matches
from analysis.controls.groups_description import make_summary_table

from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb


Loading BokehJS ...

In [2]:
hl_init(log=f'/net/archive/groups/plggneuromol/gosborcz/hail-{hail_log_uuid}.log')

2022-08-17 19:29:16 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-08-17 19:29:17 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.3, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-08-17 19:29:17 WARN  SparkConf:69 - Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


Running on Apache Spark version 3.1.2
SparkUI available at http://p0665.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /net/archive/groups/plggneuromol/gosborcz/hail-554764c8-0b17-4911-b93b-00381c2319dc.log


### 1. This analysis starts from a tsv file form the ukbiobank (described in preprocessing/downloads)

In [19]:
ht = load_annotations(wd+'data/ukb-annotations.ht')
codes = hl.import_table(wd+'raw/dataset/data-codings/coding19.tsv', delimiter='\t')

2022-05-24 21:13:22 Hail: INFO: Reading table without type imputation
  Loading field 'coding' as type str (not specified)
  Loading field 'meaning' as type str (not specified)
  Loading field 'node_id' as type str (not specified)
  Loading field 'parent_id' as type str (not specified)
  Loading field 'selectable' as type str (not specified)


### 2. Annotate the table with diagnostic codes:

additionaly:

    - only samples that have WES are kept [note: should later be changed for WGS]
    
    - withrawed samples are removed

In [20]:
ht = annotate_adr_patients(annots = ht, overwrite=True)

2022-05-24 21:13:31 Hail: INFO: wrote table with 200603 rows in 24 partitions to /localfs/23334010/annots_200k.ht


#### How many controls vs cases?

In [21]:
ht.aggregate(hl.agg.counter(ht.group))

frozendict({'adr': 602, 'control': 200001})

### 3. Annotate the table with a dictionary containing dates and diagnostic codes

Here two fields are added: date_code_zip and date_code_dict. These are used to manually explore the data.

In [22]:
ht = annotate_date_codes_dict(ht)

### 4. Annotate the table with antidepressant toxicity relevant phenotypes:

#### Intent column:

x 'therapy' - with therapeutic dose, no toxicity codes for ADs NO history od self-harm with drugs

x 'accidental' - accidental toxicity may be mixed with other drugs NO history od self-harm with drugs

x 'intentional' - any toxicity codes with any self harm history with drugs

#### Self-harm history:

x 'no_self_harm' - no codes related with self-harm (Z915, X6, X7, X80, X81, X82, X83, X84)

x 'with_drugs' - any code with 'X6'

x 'without_drugs' - other self harm codes

#### Two other columns based on most common codes:

x mental_health_inpatient:

    # depression F329, F339, F322, F321, F323, F331, F334, F251, F315, F338, F328, F313, F330, F316, F332
    
    # anxiety  F419, F410, F411, F418, F606, F408, F409, F413
    
    # both F412 or depression & anxiety codes

x drug_abuse:

    # Z864
    
#### Death column:
 
     # from ADR
     
     # different cause
     
     # still allive


In [23]:
ht = annotate_intent(ht)
ht.aggregate(hl.agg.group_by(ht.intention, hl.agg.counter(ht.self_harm)))

frozendict({'accidental': frozendict({'no_self_harm': 33, 'without_drugs': 4}), 'control': frozendict({'no_self_harm': 198673, 'with_drugs': 934, 'without_drugs': 394}), 'intentional': frozendict({'with_drugs': 471}), 'therapeutic': frozendict({'no_self_harm': 79, 'without_drugs': 5}), 'unspecified': frozendict({'no_self_harm': 9, 'without_drugs': 1})})

In [24]:
ht.aggregate(hl.agg.group_by(ht.group, hl.agg.counter(ht.death)))

frozendict({'adr': frozendict({'adr': 11, 'alive': 497, 'other': 94}), 'control': frozendict({'alive': 187625, 'other': 12376})})

### Example exploratory question:
    what are the most common diagnoses in the study group ?

In [25]:
results = ht.filter(ht.group == 'adr').explode('f_41270')
most_common_codes = results.aggregate(hl.agg.counter(results['f_41270']))

for w in sorted(most_common_codes, key=most_common_codes.get, reverse=True):
        if w not in ['Y490', 'Y491', 'Y492', 'T430', 'T431', 'T432', None]:
                r = codes.filter(codes.coding == w).meaning.collect()
                print(r, most_common_codes[w])
        if most_common_codes[w] < 100:
            break

['F32.9 Depressive episode, unspecified'] 440
['X61.9 Unspecified place'] 264
['X61.0 Home'] 259
['I10 Essential (primary) hypertension'] 257
['Z91.5 Personal history of self-harm'] 242
['Z86.4 Personal history of psychoactive substance abuse'] 197
['T39.1 4-Aminophenol derivatives'] 189
['T51.0 Ethanol'] 186
['F41.9 Anxiety disorder, unspecified'] 170
['F17.1 Harmful use'] 156
['J45.9 Asthma, unspecified'] 141
['E78.0 Pure hypercholesterolaemia'] 137
['R69 Unknown and unspecified causes of morbidity'] 134
['X60.0 Home'] 130
['T42.4 Benzodiazepines'] 124
['X60.9 Unspecified place'] 124
['K44.9 Diaphragmatic hernia without obstruction or gangrene'] 122
['R07.4 Chest pain, unspecified'] 112
['K59.0 Constipation'] 111
['Z86.7 Personal history of diseases of the circulatory system'] 110
['K21.9 Gastro-oesophageal reflux disease without oesophagitis'] 108
['T42.6 Other antiepileptic and sedative-hypnotic drugs'] 105
['Z87.1 Personal history of diseases of the digestive system'] 105
['K57.3 

In [26]:
results = ht.filter(ht.group == 'control').explode('f_41270')
most_common_codes = results.aggregate(hl.agg.counter(results['f_41270']))

for w in sorted(most_common_codes, key=most_common_codes.get, reverse=True):
        if w not in ['Y490', 'Y491', 'Y492', 'T430', 'T431', 'T432', None]:
                r = codes.filter(codes.coding == w).meaning.collect()
                print(r, most_common_codes[w])
        if most_common_codes[w] < 5000:
            break

['I10 Essential (primary) hypertension'] 57240
['E78.0 Pure hypercholesterolaemia'] 26493
['Z86.4 Personal history of psychoactive substance abuse'] 26384
['K57.3 Diverticular disease of large intestine without perforation or abscess'] 21659
['K44.9 Diaphragmatic hernia without obstruction or gangrene'] 20259
['J45.9 Asthma, unspecified'] 18275
['K21.9 Gastro-oesophageal reflux disease without oesophagitis'] 15581
['Z92.2 Personal history of long-term (current) use of other medicaments'] 15535
['E11.9 Without complications'] 14777
['Z86.7 Personal history of diseases of the circulatory system'] 14311
['R07.4 Chest pain, unspecified'] 14184
['Z87.1 Personal history of diseases of the digestive system'] 14014
['M17.9 Gonarthrosis, unspecified'] 12920
['Z88.0 Personal history of allergy to penicillin'] 12840
['H26.9 Cataract, unspecified'] 12835
['I25.1 Atherosclerotic heart disease'] 12521
['E66.9 Obesity, unspecified'] 12286
['Z92.1 Personal history of long-term (current) use of anticoa

### 5. Table export after the initial phenotyping step:

In [27]:
ht.write('/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/data/hospital-codes-phenotyped.ht')

2022-05-24 21:24:12 Hail: INFO: wrote table with 200603 rows in 24 partitions to /net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/data/hospital-codes-phenotyped.ht


### 6. Remove related individuals from the final table

In [3]:
ht = hl.read_table('/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/data/hospital-codes-phenotyped.ht')

In [ ]:
ht = remove_related(ht, kin_cut=0.125)

### 7. For each case find a PCA match
we chose either 6 or 17 PCs (based on this paper: https://www.biorxiv.org/content/10.1101/841452v1.full)

In [ ]:
ht = anno_matches(ht)

#### export final annotations for burden analysis and save the table

In [ ]:
to_export = ht.select(
    ht.group,
    ht.intention,
    ht.self_harm,
    ht.drug_abuse,
    ht.mental_health_inpatient,
    ht.is_match_6,
    ht.match_6,
    ht.is_match_17,
    ht.match_17
)

to_export.write('/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/data/annots-for-burden.ht')
ht.write('/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/data/full-annots-for-burden.ht')

### 8. Prepare a table to be included in the final paper:

* note: due to no major differences in all controls and pca_6 controls and for simplicity 'PCA_controls_6', and 'PCA_controls_17' are skipped in the final analyses*

x male_to_female_ratio 
x age (+/- sd)
x townsend (+/- sd)
x bmi (+/- )
x depression and/or anxiety hospital codes (mental health inpatient)
x self harm with drugs (& without drugs)
x drug abuse history
x additional fields
x number of self-reported non-cancer illnesses
x ever contemplatedcontemplated self-harm 
x substances taken for depression

In [3]:
ht = hl.read_table('/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/data/full-annots-for-burden.ht')

In [4]:
manuscript_table = make_summary_table(ht)

In [5]:
manuscript_table

,n,percentage of females,Townsend mean,Townsend SD,BMI mean,BMI SD,age when at assesment centre mean,age when at assesment centre SD,depression hospital code,anxiety hospital code,both depression and anxiety hospital code,self harm with drugs,self harm without drugs,drug abuse history,number of non-cancer illness reported mean,number of non-cancer illness reported sd,ever contemplated self harm,taken any medication for depression
0,602,59.136213,0.341858,3.626968,28.379127,6.127306,53.617940,8.212514,267,16,200,471,10,197,3.308970,2.502003,97,105
1,471,60.084926,0.560783,3.671732,28.435662,6.032138,52.125265,7.704413,230,7,158,471,0,146,3.367304,2.559465,87,88
2,121,56.198347,-0.509372,3.294889,28.223059,6.591078,59.033058,7.859230,34,9,39,0,9,49,3.066116,2.273584,10,16
3,199220,55.052706,-1.340377,3.049399,27.373099,4.752145,56.464220,8.102170,7460,3707,4015,930,391,26261,1.865460,1.864655,9718,19280
4,930,56.666667,0.348471,3.663557,28.214876,5.819613,53.523656,8.202542,363,24,187,930,0,275,2.987083,2.598717,138,137
5,15182,65.182453,-0.672354,3.347470,28.540304,5.616781,56.830655,8.090759,7460,3707,4015,574,263,4393,3.046973,2.397562,1395,2968


In [6]:
manuscript_table.to_csv(
    '/net/archive/groups/plggneuromol/gosborcz/ifpan-gosborcz-ukb/results/description-table.csv'
                       )